In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
import mlflow
from mlflow.models import infer_signature

In [5]:
from ucimlrepo import fetch_ucirepo
import numpy as np
import pandas as pd
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
import shap
from cubist import Cubist

# fetch dataset
steel_industry_energy_consumption = fetch_ucirepo(id=851)

In [6]:
df = pd.DataFrame(steel_industry_energy_consumption.data.original)

In [7]:
df['date'] = pd.to_datetime(df['date'], format='%d/%m/%Y %H:%M')

In [8]:
# Utilizamos One Hot Enconder para ["Day_of_week", "Load_Type"]  eliminando las columnas iniciales
df = pd.get_dummies(df, columns=["Day_of_week", "Load_Type"], drop_first=True)

In [9]:
# Visualizamos las columnas actuales del DataFrame
# df.columns.to_list()

In [10]:
# Hacemos una diferenciación entre fines de semana y días hábiles
df['IsWeekend'] = df['WeekStatus'] == 'Weekend'

In [11]:
# Eliminamos Date porque según EDA la potencia tiene muy baja dependencia de la temporalidad
# Eiminamos WeekStatus porque según EDA la potencia cambia entre día laboral y fin de semana
df = df.drop(['date', 'WeekStatus'], axis=1)

In [12]:
#df.dtypes

In [13]:
mlflow.set_tracking_uri(uri="http://localhost:5001")

In [14]:
# Definimos conjunto de entrenamiento y variable dependiente Y
X = df
X = df.drop('Usage_kWh', axis=1)
y = df['Usage_kWh']

In [15]:
experiment_name = "RegresionLineal"
experiment_id = mlflow.create_experiment(experiment_name) if not mlflow.get_experiment_by_name(experiment_name) else mlflow.get_experiment_by_name(experiment_name).experiment_id

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [17]:
with mlflow.start_run(experiment_id=experiment_id):
    
    # Desarrollamos Regresión Lineal   
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Crear el modelo de regresión lineal
    model = LinearRegression()
    
    # Entrenar el modelo
    model.fit(X_train, y_train)
    

In [18]:
mlflow.sklearn.log_model(model, "model")

# Realizar predicciones
y_pred = model.predict(X_test)

# Evaluar el modelo
# Calcular MSE
mse = mean_squared_error(y_test, y_pred)

# Calcular RMSE
rmse = np.sqrt(mse)

# Calcular MAE
mae = mean_absolute_error(y_test, y_pred)

# Calcular R^2
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error (MSE): {mse}')
print(f'Root Mean Squared Error (RMSE): {rmse}')
print(f'Mean Absolute Error (MAE): {mae}')
print(f'R^2 Score: {r2}')

mlflow.log_param("mse", mse)
mlflow.log_param("rmse", rmse)
mlflow.log_param("mae", mae)
mlflow.log_metric("r2", r2)

Mean Squared Error (MSE): 17.842279082761355
Root Mean Squared Error (RMSE): 4.224012202013786
Mean Absolute Error (MAE): 2.5337743872306184
R^2 Score: 0.9843037490428197


In [1]:
# Example:
mlflow.log_param("learning_rate", learning_rate)  # If you used a learning rate parameter

# Example (if relevant):
mlflow.log_artifact("data.csv", "data")  # Assuming data.csv is used during training

NameError: name 'mlflow' is not defined

Agregamos modelos a utilizar, Arbol de Decisión, RandomForest, XGBoost Regressor, CUBIST

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb

modelos = [('LinearRegression',LinearRegression()),
    ('DecisionTreeRegressor', DecisionTreeRegressor(max_depth=2)),
    ('RandomForestRegressor', RandomForestRegressor(min_samples_split=2, min_weight_fraction_leaf=0.0,n_estimators=40, n_jobs=-1, oob_score=True,random_state=None, verbose=0, warm_start=False)),
    ('XGBRegressor', xgb.XGBRegressor(learning_rate=0.01,n_estimators=500,max_depth=5,eval_metric='rmsle')),
    ('Cubist', Cubist(n_rules=500,neighbors=None,unbiased=True,auto=False,extrapolation=0.1,n_committees=5))
]


#procedimiento para comparar los modelos
for nombre, model in modelos:
  model.fit(X_train, y_train)
  y_pred = model.predict(X_test)
  mse = mean_squared_error(y_test, y_pred)
  rmse = np.sqrt(mse)
  mae = mean_absolute_error(y_test, y_pred)
  r2 = r2_score(y_test, y_pred)

  print(f'{nombre}," score: ",{model.score(X_test,y_pred):.03f}',end=" ")
  print(f'Mean Squared Error (MSE): {mse:.03f}',end=" ")
  print(f'Root Mean Squared Error (RMSE): {rmse:.03f}',end=" ")
  print(f'Mean Absolute Error (MAE): {mae:.03f}',end=" ")
  print(f'R^2 Score: {r2:.03f}',end=" ")
  print("")

Buscamos mejores valores de hiperparámetros para los 2 modelos con mejores resultados, XGBRegressor y Cubist
1. XGBRegressor

In [ ]:
from pprint import pprint
import numpy as np

n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_leaves = [0, 8, 10, 12, 16, 20]  # 0 equivale a 'no limit'
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)] + [None]


random_grid = {'n_estimators': n_estimators,
               'max_leaves': max_leaves,
               'max_depth': max_depth}

print('Los valores a probar en la búsqueda aleatoria son:')
pprint(random_grid)

print()
print('El número total de combinaciones de parámetros de entrenamiento es',
      len(random_grid['n_estimators']) *
      len(random_grid['max_leaves']) *
      len(random_grid['max_depth'])
      )

Usaremos la recomendación de busqueda con el 10%

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

xgbb = xgb.XGBRegressor()
xgb_random = RandomizedSearchCV(estimator = xgbb,
                               param_distributions = random_grid,
                               n_iter = 20,
                               cv = 3,          # Validación cruzada 3-fold
                               verbose=2,
                               random_state=0,
                               n_jobs = -1      # Paralelizar en todos los cores disponibles
                               )
xgb_random.fit(X_train, y_train)

In [ ]:
xgb_random_best = xgb_random.best_estimator_

print('Los hiperparámetros del mejor modelo son:')
pprint(xgb_random.best_params_)
print()

print('Exactitud luego de búsqueda aleatoria en entrenamiento:', xgb_random_best.score(X_train, y_train))
print('Exactitud luego de búsqueda aleatoria en validación:', xgb_random_best.score(X_test, y_test))

2. Búsqueda de hiperparámetros para el método Cubist

In [ ]:
from pprint import pprint
import numpy as np

###Hiperparámetros a considerar
# n_rules (int, default=500)
# n_committees (int, default=0):
# neighbors (int, default=None)
# extrapolation (float, default=0.05):

n_rules = [int(x) for x in np.linspace(start = 200, stop = 1000, num = 10)]
n_committees = [1, 2, 5, 10, 15]  # 5 es el recomendado en la documentación
neighbors = [int(x) for x in np.linspace(1, 9, num = 1)]
extrapolation = [0.01, 0.03, 0.05, 0.07, 0.09] #0,05 = 5% es el recomendado


random_grid = {'n_rules': n_rules,
               'n_committees': n_committees,
               'neighbors': neighbors,
               'extrapolation':extrapolation}

print('Los valores a probar en la búsqueda aleatoria son:')
pprint(random_grid)

print()
print('El número total de combinaciones de parámetros de entrenamiento es',
      len(random_grid['n_rules']) *
      len(random_grid['n_committees']) *
      len(random_grid['neighbors']) *
      len(random_grid['extrapolation'])
      )

In [ ]:
#usamos la recomendación de probar con el 10%

cbst = Cubist()
cbst_random = RandomizedSearchCV(estimator = cbst,
                               param_distributions = random_grid,
                               n_iter = 5,
                               cv = 3,          # Validación cruzada 3-fold
                               verbose=2,
                               random_state=0,
                               n_jobs = -1      # Paralelizar en todos los cores disponibles
                               )
cbst_random.fit(X_train, y_train)

In [ ]:
cbst_random_best = cbst_random.best_estimator_

print('Los hiperparámetros del mejor modelo son:')
pprint(cbst_random.best_params_)
print()

print('Exactitud luego de búsqueda aleatoria en entrenamiento:', cbst_random_best.score(X_train, y_train))
print('Exactitud luego de búsqueda aleatoria en validación:', cbst_random_best.score(X_test, y_test))

**Uso de la librería OPTUNA para encontrar los mejores hiperparámetros**

Estimamos los mejores hiperparámetros usando la librería OPTUNA sugerida en el plantemiento del proyecto

In [ ]:
import optuna
import optuna.visualization as vis

# Para trabajar con optuna, debemos definir la función objetivo que luego le diremos a optuna que debe minimizar (según nuestro indicador escogido RMSE)
def objective(trial):
  # Dejamos que optuna sugiera los hiperparámetros iniciales
  n_rules = trial.suggest_int("n_rules", 200, 1000)
  n_committees = trial.suggest_int("n_committees", 1, 15)
  neighbors = trial.suggest_int("neighbors", 1, 9)
  extrapolation = trial.suggest_float("extrapolation",0.01, 0.09)

  #Definimos el modelo Cubist a utilizar para la optimización
  cbst = Cubist(n_rules= n_rules, n_committees=n_committees, neighbors=neighbors, extrapolation=extrapolation)
  cbst.fit(X_train, y_train)

  # Make predictions and calculate RMSE
  y_pred = model.predict(X_test)
  rmse = np.sqrt(mean_squared_error(y_test, y_pred))
  mae = mean_absolute_error(y_test, y_pred)
  r2 = r2_score(y_test, y_pred)

  # Return MAE
  return mae

In [ ]:
# Creamos el "objeto de estudio", como llama Optuna
study = optuna.create_study(direction="minimize")

# Ejecutamos el proceso en si de optimización, le pasamos la función objetivo definida en el paso previo
study.optimize(objective, n_trials=10, show_progress_bar=True)

In [2]:
# Imprimimos los resultados del estudio con optuna
print("Best trial:", study.best_trial)
print("Best hyperparameters:", study.best_params)

NameError: name 'study' is not defined

In [ ]:
#Utilizamos algunos gráficos utiles de la librería optuna
# Plotear el historial de la optimización realizada
vis.plot_optimization_history(study)

# Plotear tipo 'slice'
#vis.plot_slice(study, params=["n_rules", "n_committees"])

# Ploteo tipo 'contorno'
#vis.plot_contour(study, params=["neighbors", "extrapolation"])

# Plotear parallel_coordinate
vis.plot_parallel_coordinate(study)

In [ ]:
optuna.visualization.plot_optimization_history(study)